In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as transforms
import torchvision.models as models

from skimage import io, transform
import numpy as np
import pandas as pd
import os
from PIL import Image
import random

In [71]:
NUM_EPOCHS = 10
LR = 1e-5
BATCH_SIZE = 4

In [72]:
tfms = transforms.Compose([transforms.Resize((256,256)), transforms.CenterCrop((224,224)),
                           transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [73]:
categories = pd.read_csv("/home/krypt/myStuff/skillenza/paniithackathon2019/training.5k/training/solution.csv")["category"]
def getitem(idx):
    img_name = os.path.join("/home/krypt/myStuff/skillenza/paniithackathon2019/training.5k/training/training/", str(idx+1) + ".png")
    image = Image.open(img_name)
    if tfms:
        image = tfms(image)
    category = categories[idx]
    sample = {'image': image, 'category': category}

    return sample

In [74]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [75]:
model = models.resnet34(pretrained=True)

model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(512, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.LogSoftmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [76]:
print(model.children)

count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

<bound method Module.children of Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)


In [80]:
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    accuracy = 0
    count = 0

    optimizer.zero_grad()
    for c in range(1,1000):
        if c%BATCH_SIZE == 0:
            optimizer.step()
            optimizer.zero_grad()

        try:
            data = getitem(random.randint(0,4999))
        except:
            continue
        
        images = data["image"].unsqueeze(0)
        targets = torch.tensor([np.asscalar(data["category"])]).long()-1
        
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
                
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1) == targets)
        count += 1

        if c%200 == 0:
            print("Epoch:", epoch, "Iter:", c, "Total loss:", total_loss, "Accuracy:", accuracy.cpu().numpy()/count)
    
    
    print("Epoch:", epoch, "Total loss:", total_loss)
    print("Epoch:", epoch, "Accuracy:", accuracy.cpu().numpy()/count)    

/home/krypt/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 0 Iter: 200 Total loss: tensor(40.8865, device='cuda:0') Accuracy: 0.925
Epoch: 0 Iter: 400 Total loss: tensor(70.1950, device='cuda:0') Accuracy: 0.9275
Epoch: 0 Iter: 600 Total loss: tensor(81.6874, device='cuda:0') Accuracy: 0.9466666666666667
Epoch: 0 Iter: 800 Total loss: tensor(100.4772, device='cuda:0') Accuracy: 0.9525
Epoch: 0 Total loss: tensor(127.4168, device='cuda:0')
Epoch: 0 Accuracy: 0.955955955955956
Epoch: 1 Iter: 200 Total loss: tensor(9.4006, device='cuda:0') Accuracy: 0.99
Epoch: 1 Iter: 400 Total loss: tensor(35.2746, device='cuda:0') Accuracy: 0.9825
Epoch: 1 Iter: 600 Total loss: tensor(53.3477, device='cuda:0') Accuracy: 0.98
Epoch: 1 Iter: 800 Total loss: tensor(74.6654, device='cuda:0') Accuracy: 0.97875
Epoch: 1 Total loss: tensor(85.1705, device='cuda:0')
Epoch: 1 Accuracy: 0.977977977977978
Epoch: 2 Iter: 200 Total loss: tensor(32.4623, device='cuda:0') Accuracy: 0.9597989949748744
Epoch: 2 Iter: 400 Total loss: tensor(63.0259, device='cuda:0') Accu